## Import Packages


In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

## Function to Scrape BB Reference data

In [45]:
def BBRef_player_stats_scraper(year_start, year_end):
    
    # create an empty list for the combined stats
    complete_stats = []
    
    # assign the year to start loop at
    year = year_start
    
    #number of years after the first year to be considered (add one to compensate for range function)
    delta = year_end - year_start + 1
    
    # start loop
    for x in range(delta):
        
        # URL page function is scraping from
        url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html".format(year)
        
        # HTML for above url
        html = urlopen(url)
        soup = BeautifulSoup(html)
        
        # use findALL() to get the column headers
        soup.findAll('tr', limit=2)
        
        # use getText()to extract the text into a list
        headers = [th.getText() for th in soup.findAll('tr', limit=2)[0].findAll('th')]
        
        # exclude the first column as we will not need the ranking order from Basketball Reference for the analysis
        headers = headers[1:]
        # avoid the first header row
        rows = soup.findAll('tr')[1:]
        player_stats = [[td.getText() for td in rows[i].findAll('td')]
                        for i in range(len(rows))]
        
        #loop through players adding year for stats and then appending each player to the complete list
        for player in player_stats:
            player.insert(0, year)
            complete_stats.append(player)
        
        # move to the next year and loop again
        year = year + 1
    
    # add in the year column header
    headers.insert(0,'year')
    
    # create a dataframe from the complete list
    stats = pd.DataFrame(complete_stats, columns = headers)
    
    return stats
    

## Run Function

In [46]:

# Stats for years
df = BBRef_player_stats_scraper(2016, 2020)

## Export to CSV

In [51]:
df.to_csv(bbref_df.csv)

,year,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,2016,Quincy Acy,PF,25,SAC,59,29,14.8,2.0,3.6,...,.735,1.1,2.1,3.2,0.5,0.5,0.4,0.5,1.7,5.2
1,2016,Jordan Adams,SG,21,MEM,2,0,7.5,1.0,3.0,...,.600,0.0,1.0,1.0,1.5,1.5,0.0,1.0,1.0,3.5
2,2016,Steven Adams,C,22,OKC,80,80,25.2,3.3,5.3,...,.582,2.7,3.9,6.7,0.8,0.5,1.1,1.1,2.8,8.0
3,2016,Arron Afflalo,SG,30,NYK,71,57,33.4,5.0,11.3,...,.840,0.3,3.4,3.7,2.0,0.4,0.1,1.2,2.0,12.8
4,2016,Alexis Ajinça,C,27,NOP,59,17,14.6,2.5,5.3,...,.839,1.3,3.3,4.6,0.5,0.3,0.6,0.9,2.3,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3316,2020,Trae Young,PG,21,ATL,60,60,35.3,9.1,20.8,...,.860,0.5,3.7,4.3,9.3,1.1,0.1,4.8,1.7,29.6
3317,2020,Cody Zeller,C,27,CHO,58,39,23.1,4.3,8.3,...,.682,2.8,4.3,7.1,1.5,0.7,0.4,1.3,2.4,11.1
3318,2020,Tyler Zeller,C,30,SAS,2,0,2.0,0.5,2.0,...,,1.5,0.5,2.0,0.0,0.0,0.0,0.0,0.0,1.0
3319,2020,Ante Žižić,C,23,CLE,22,0,10.0,1.9,3.3,...,.737,0.8,2.2,3.0,0.3,0.3,0.2,0.5,1.2,4.4
